## Running an agent from Our Registry (AI Foundry)

In [24]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder

project = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint="https://agent-service-demo.services.ai.azure.com/api/projects/agent-service-demo-project")


In [25]:
QUERY="How do I enable FedEx Electronic Trade Documents in FedEx Ship Manager Software?"

In [26]:
agent = project.agents.get_agent("asst_qLI4yveCYhur62Ya4QhEkvFz")

thread = project.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

message = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=QUERY
)

run = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    messages = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

    for message in messages:
        if message.text_messages:
            print(f"{message.role}: {message.text_messages[-1].text.value}")

Created thread, ID: thread_V40cgPabFSfzXXyw5hHmo0EX
MessageRole.USER: How do I enable FedEx Electronic Trade Documents in FedEx Ship Manager Software?
MessageRole.AGENT: To enable FedEx Electronic Trade Documents in FedEx Ship Manager Software, follow these steps:

1. **Contact a FedEx Account Executive**: Initially, you need to contact a FedEx account executive to enable the FedEx Electronic Trade Documents feature. If you do not have a FedEx account executive, proceed with the steps below.

2. **Access System Settings**:
   - Select "System Settings" from the "Customize" drop-down menu, which will display the System Settings – System/Account screen.
   
3. **Modify System Settings**:
   - Select the appropriate system number and click "Modify" to open the System Settings screen.

4. **Activate Electronic Trade Documents**:
   - Click on the "1-Your System Settings" tab.
   - In the Trade Documents section at the bottom of the screen, select the "Activate Electronic Trade Documents" c

## Adding Tracing to Monitor Agent Performance

To monitor and debug your agent interactions, we'll add OpenTelemetry tracing that integrates with Azure AI Foundry's tracing capabilities.

In [27]:
# Install required packages for tracing (run once)
# !pip install azure-monitor-opentelemetry opentelemetry-instrumentation-openai-v2

# Configure tracing for Azure AI Foundry
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor
from opentelemetry import trace

# Get the connection string for Application Insights
connection_string = project.telemetry.get_application_insights_connection_string()
print(f"Configured tracing with Application Insights")

# Configure Azure Monitor and instrument OpenAI SDK
configure_azure_monitor(connection_string=connection_string)
OpenAIInstrumentor().instrument()

# Get tracer for custom spans
tracer = trace.get_tracer(__name__)

Configured tracing with Application Insights


In [28]:
# Agent interaction with tracing enabled
def run_agent_with_tracing(user_message: str, span_id: str = "agent_conversation"):
    with tracer.start_as_current_span(span_id) as current_span:
        # Get current span to add custom attributes
        current_span.set_attribute("agent.id", "asst_qLI4yveCYhur62Ya4QhEkvFz")
        current_span.set_attribute("user.message", user_message)
        current_span.set_attribute("span.custom_id", span_id)
        
        # Get agent
        agent = project.agents.get_agent("asst_qLI4yveCYhur62Ya4QhEkvFz")
        current_span.set_attribute("agent.name", getattr(agent, 'name', 'ElectronicTradeExpert'))
        
        # Create thread
        thread = project.agents.threads.create()
        print(f"Created thread, ID: {thread.id}")
        current_span.set_attribute("thread.id", thread.id)
        
        # Create message
        message = project.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_message
        )
        
        # Run agent
        with tracer.start_as_current_span("agent_processing") as processing_span:
            processing_span.set_attribute("operation.type", "agent_run")
            
            run = project.agents.runs.create_and_process(
                thread_id=thread.id,
                agent_id=agent.id
            )
            
            processing_span.set_attribute("run.id", run.id)
            processing_span.set_attribute("run.status", run.status)
        
        # Handle results
        if run.status == "failed":
            current_span.set_attribute("error.occurred", True)
            current_span.set_attribute("error.message", str(run.last_error))
            print(f"Run failed: {run.last_error}")
            return None
        else:
            current_span.set_attribute("error.occurred", False)
            messages = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
            
            conversation_log = []
            for message in messages:
                if message.text_messages:
                    message_content = message.text_messages[-1].text.value
                    conversation_log.append(f"{message.role}: {message_content}")
                    print(f"{message.role}: {message_content}")
            
            # Add conversation summary to span
            current_span.set_attribute("conversation.message_count", len(conversation_log))
            
            return conversation_log

# Run the agent with tracing
conversation = run_agent_with_tracing(QUERY, "ETD_Initial_Query")

Created thread, ID: thread_Fme4c8IVUvJv81iqdN057Ktp
MessageRole.USER: How do I enable FedEx Electronic Trade Documents in FedEx Ship Manager Software?
MessageRole.AGENT: To enable FedEx Electronic Trade Documents in FedEx Ship Manager Software, follow these steps:

1. **Select System Settings:** From the Customize drop-down menu, select System Settings. This will display the System Settings – System/Account screen.

2. **Modify Settings:** Select the appropriate system number and click Modify. 

3. **Your System Settings Tab:** Click on the 1-Your System Settings tab.

4. **Activate Electronic Trade Documents:** In the Trade Documents section at the bottom of the screen, select the Activate Electronic Trade Documents check box. This will display the Electronic Trade Documents – Terms and Conditions screen.

5. **Accept Terms and Conditions:** Click the I accept the terms and conditions check box and then click OK.

6. **Set Communications Settings:** In the Communications Settings area

## Viewing Your Traces

After running the agent code above, you can view the traces in Azure AI Foundry:

1. **Go to Azure AI Foundry Portal** → Your Project → **Tracing** (in left navigation)
2. **Find Your Traces**: Look for traces with:
   - `agent_conversation` spans showing the full interaction
   - `agent_processing` spans showing the agent execution
   - Custom attributes like `agent.id`, `thread.id`, `user.message`
   - Performance metrics and timing information

3. **Trace Information Captured**:
   - **Agent Details**: Agent ID and name
   - **User Input**: The message sent to the agent
   - **Thread Management**: Thread creation and ID
   - **Processing Status**: Success/failure status
   - **Performance**: Execution time and latency
   - **Error Handling**: Error details if any occurred

4. **Benefits of Tracing**:
   - **Debug Issues**: See exactly where problems occur
   - **Performance Monitoring**: Track response times
   - **Usage Analytics**: Understand agent interaction patterns
   - **Quality Assurance**: Monitor agent response quality over time

## Test Cases for FedEx ETD Agent

Here are 3 key test questions to evaluate your agent's performance:

In [29]:
# Test Case 1: ETD Activation and Setup
print("🧪 TEST 1: ETD System Activation")
print("Question: How do I enable FedEx Electronic Trade Documents in FedEx Ship Manager Software?")
print("Expected: Step-by-step activation process including system settings and terms acceptance")
print("-" * 60)

test1_result = run_agent_with_tracing("How do I enable FedEx Electronic Trade Documents in FedEx Ship Manager Software?", "ETD_Activation_Test")

print("\n" + "="*80 + "\n")

# Test Case 2: Upload Options and Settings
print("🧪 TEST 2: Upload Configuration")
print("Question: What's the difference between 'At time of shipment' vs 'At regular hourly upload time (in batch)' for ETD uploads?")
print("Expected: Detailed comparison of both options with pros/cons and when to use each")
print("-" * 60)

test2_result = run_agent_with_tracing("What's the difference between 'At time of shipment' vs 'At regular hourly upload time (in batch)' for ETD uploads?", "ETD_Upload_Options_Test")

print("\n" + "="*80 + "\n")

# Test Case 3: Letterhead and Signature Requirements
print("🧪 TEST 3: Document Formatting Requirements")
print("Question: What are the image requirements for uploading company letterhead and signature in ETD?")
print("Expected: Specific dimensions, file formats, and application settings for letterhead/signature images")
print("-" * 60)

test3_result = run_agent_with_tracing("What are the image requirements for uploading company letterhead and signature in ETD?", "ETD_Image_Requirements_Test")

🧪 TEST 1: ETD System Activation
Question: How do I enable FedEx Electronic Trade Documents in FedEx Ship Manager Software?
Expected: Step-by-step activation process including system settings and terms acceptance
------------------------------------------------------------
Created thread, ID: thread_RJ5X5xVdkZnpYFNmmStR0Z9g
Created thread, ID: thread_RJ5X5xVdkZnpYFNmmStR0Z9g
MessageRole.USER: How do I enable FedEx Electronic Trade Documents in FedEx Ship Manager Software?
MessageRole.AGENT: To enable FedEx Electronic Trade Documents in FedEx Ship Manager Software, follow these steps:

1. **Contact Your FedEx Account Executive**: It's essential to have your FedEx account executive guide you in enabling this feature. If you don't have one, proceed with the steps below.

2. **Access System Settings**: 
   - Open the FedEx Ship Manager Software.
   - Select "System Settings" from the "Customize" drop-down menu to display the System Settings – System/Account screen.

3. **Modify System Setti